In [215]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import datetime
import math, time
import itertools
from sklearn import preprocessing
import datetime
from operator import itemgetter
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from math import sqrt
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn.grid_search import GridSearchCV   #Perforing grid search

In [216]:
def get_stock_data(stock_name, normalized=0):
    
#     url="http://www.google.com/finance/historical?q=%s&ei=u-lHWfGPNNWIsgHHqIqICw&output=csv" % stock_name 
    url="http://www.google.com/finance/historical?q="+stock_name+"&startdate=Aug+03%2C+2015&enddate=Aug+02%2C+2017&num=30&ei=d7d4WbrtDYq1swH0mp-4CA&output=csv" 
#     print(url)
    col_names = ['Date','Open','High','Low','Close','Volume']
    stock = pd.read_csv(url, header=0, names=col_names) 
    df = pd.DataFrame(stock)

    df["Volume"] = df["Volume"] / 10000
#     df.drop(df.columns[[3,5]], axis=1, inplace=True) 
    return df


def shift(seq, n):
    n = n % len(seq)
    return seq[n:] + seq[:n] 

def get_rsi_timeseries(prices, n=14):
    deltas = (prices-prices.shift(1)).fillna(0)
    avg_of_gains = deltas[1:n+1][deltas > 0].sum() / n
    avg_of_losses = -deltas[1:n+1][deltas < 0].sum() / n

    # Set up pd.Series container for RSI values
    rsi_series = pd.Series(0.0, deltas.index)

    # Now calculate RSI using the Wilder smoothing method, starting with n+1 delta.
    up = lambda x: x if x > 0 else 0
    down = lambda x: -x if x < 0 else 0
    i = n+1
    for d in deltas[n+1:]:
        avg_of_gains = ((avg_of_gains * (n-1)) + up(d)) / n
        avg_of_losses = ((avg_of_losses * (n-1)) + down(d)) / n
        if avg_of_losses != 0:
            rs = avg_of_gains / avg_of_losses
            rsi_series[i] = 100 - (100 / (1 + rs))
        else:
            rsi_series[i] = 100
        i += 1

    return rsi_series

def get_stock_data_analysis(stock_df):  
    first_ma = 12
    second_ma = 26
    stock_df = stock_df[::-1]
    data = [round(x,6) for x in stock_df["Close"]]
    
    new_df = pd.DataFrame(stock_df["Date"])
    new_df["Close"] = data
    
    new_df["C_1"] = shift(data,-1)
    new_df["C_2"] = shift(data,-2)
    new_df["C_3"] = shift(data,-3)
    new_df["C_4"] = shift(data,-4)
    new_df["C_5"] = shift(data,-5)
    new_df["C_6"] = shift(data,-6)
    new_df["C_7"] = shift(data,-7)
    new_df["C_8"] = shift(data,-8)
    new_df["C_9"] = shift(data,-9)
    new_df["C_10"] = shift(data,-10)
    
#     new_df["C1_pct"] =  round(new_df["Close"] / new_df["C_1"],6)
#     new_df["C2_pct"] =  round(new_df["C_1"] / new_df["C_2"],6)
#     new_df["C3_pct"] =  round(new_df["C_2"] / new_df["C_3"],6)
#     new_df["low_high_ratio"] = round(new_df["High"] / new_df["Low"],6)    
#     new_df["open_close_ratio"] = round(new_df["Close"] / new_df["Open"],6)
#     new_df["ma_ratio"] = round(new_df["Close"].rolling(window=first_ma).mean() / new_df["Close"].rolling(window=second_ma).mean(),6)
    
    ##############################
    
    first_ma_name = "ma%s" % first_ma
    second_ma_name = "ma%s" % second_ma
#     ma_relations = first_ma_name +"_"+ second_ma_name
    
#     new_df[ma_relations] = np.where(new_df["Close"].rolling(window=first_ma).mean() > new_df["Close"].rolling(window=second_ma).mean(),1,-1)
#     new_df["ma_change"] = round(new_df[ma_relations] - new_df[ma_relations].rolling(window=2).mean(),6)
    
    ##############################
    
#     new_df['ave'], new_df['upper'], new_df['lower'] = bbands(new_df["Close"], length=n, numsd=1)
#     new_df["dd"] = dd(stock_df["Close"],5)
#     new_df["max_dd"] = max_dd(stock_df["Close"])
#     new_df["RSI"] = round(get_rsi_timeseries(stock_df["Close"]),6)
    a = np.array(new_df)
#     new_df["RSI"] = a[:,new_df.columns.get_loc("RSI")][::-1]
#     new_df = new_df[::-1]
    new_df["Open"] = stock_df["Open"]    
    new_df["High"] = stock_df["High"]
    new_df["Volume"] = stock_df["Volume"]

    
    new_df = new_df[::-1]
    return new_df

In [217]:
stock_name = 'BLPH'
df_full = get_stock_data(stock_name,0)
df_full.head()

,Date,Open,High,Low,Close,Volume
0,2-Aug-17,1.25,1.30,1.21,1.22,23.4557
1,1-Aug-17,1.29,1.30,1.20,1.26,30.0567
2,31-Jul-17,1.30,1.32,1.25,1.30,31.1901
3,28-Jul-17,1.30,1.35,1.27,1.30,35.0260
4,27-Jul-17,1.42,1.44,1.32,1.33,43.4994


In [218]:
df = get_stock_data_analysis(df_full)
df.head()

,Date,Close,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,C_10,Open,High,Volume
0,2-Aug-17,1.22,1.26,1.30,1.30,1.33,1.43,1.39,1.48,1.51,1.53,1.54,1.25,1.30,23.4557
1,1-Aug-17,1.26,1.30,1.30,1.33,1.43,1.39,1.48,1.51,1.53,1.54,1.50,1.29,1.30,30.0567
2,31-Jul-17,1.30,1.30,1.33,1.43,1.39,1.48,1.51,1.53,1.54,1.50,1.56,1.30,1.32,31.1901
3,28-Jul-17,1.30,1.33,1.43,1.39,1.48,1.51,1.53,1.54,1.50,1.56,1.47,1.30,1.35,35.0260
4,27-Jul-17,1.33,1.43,1.39,1.48,1.51,1.53,1.54,1.50,1.56,1.47,1.42,1.42,1.44,43.4994


In [219]:
window = 1
gap = 1.01
df["Close_shift"] = df["Close"].shift(window)
df["Shift_ratio"] = round(df["Close"] / df["Close_shift"],5)
df["Result"] = 1*(df["Shift_ratio"] >= gap)
df.fillna(0, inplace=True)
df.head(10)

,Date,Close,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,C_10,Open,High,Volume,Close_shift,Shift_ratio,Result
0,2-Aug-17,1.22,1.26,1.30,1.30,1.33,1.43,1.39,1.48,1.51,1.53,1.54,1.25,1.30,23.4557,0.00,0.00000,0
1,1-Aug-17,1.26,1.30,1.30,1.33,1.43,1.39,1.48,1.51,1.53,1.54,1.50,1.29,1.30,30.0567,1.22,1.03279,1
2,31-Jul-17,1.30,1.30,1.33,1.43,1.39,1.48,1.51,1.53,1.54,1.50,1.56,1.30,1.32,31.1901,1.26,1.03175,1
3,28-Jul-17,1.30,1.33,1.43,1.39,1.48,1.51,1.53,1.54,1.50,1.56,1.47,1.30,1.35,35.0260,1.30,1.00000,0
4,27-Jul-17,1.33,1.43,1.39,1.48,1.51,1.53,1.54,1.50,1.56,1.47,1.42,1.42,1.44,43.4994,1.30,1.02308,1
5,26-Jul-17,1.43,1.39,1.48,1.51,1.53,1.54,1.50,1.56,1.47,1.42,1.41,1.39,1.44,14.4370,1.33,1.07519,1
6,25-Jul-17,1.39,1.48,1.51,1.53,1.54,1.50,1.56,1.47,1.42,1.41,1.42,1.48,1.51,34.4835,1.43,0.97203,0
7,24-Jul-17,1.48,1.51,1.53,1.54,1.50,1.56,1.47,1.42,1.41,1.42,1.42,1.51,1.52,13.5308,1.39,1.06475,1
8,21-Jul-17,1.51,1.53,1.54,1.50,1.56,1.47,1.42,1.41,1.42,1.42,1.45,1.53,1.54,22.8243,1.48,1.02027,1
9,20-Jul-17,1.53,1.54,1.50,1.56,1.47,1.42,1.41,1.42,1.42,1.45,1.35,1.53,1.58,27.3972,1.51,1.01325,1


In [220]:
print(df[df.Result == 1].Result.count())
print(df[df.Result == 0].Result.count())

243
262


In [221]:
col_names = np.array(df.columns)
# print(df[np.delete(col_names,[1,2,5,7])])
X = df[np.delete(col_names,[0,8,9,10])]
y = df["Result"]
features_corr_label = df.corr()['Result'].tolist()
features = features_corr_label[:-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)

In [222]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [223]:
print(col_names)
rf = model.feature_importances_
#show to plot importances
importances = pd.DataFrame({'feature':X.columns,'importance':np.round(model.feature_importances_,3)})
importances = importances.sort_values('importance',ascending=False).set_index('feature')
print(importances)
importances.plot.bar()

['Date' 'Close' 'C_1' 'C_2' 'C_3' 'C_4' 'C_5' 'C_6' 'C_7' 'C_8' 'C_9'
 'C_10' 'Open' 'High' 'Volume' 'Close_shift' 'Shift_ratio' 'Result']
             importance
feature                
Shift_ratio       0.681
Result            0.250
High              0.012
Volume            0.010
Open              0.009
Close             0.007
C_4               0.007
C_6               0.007
C_5               0.005
Close_shift       0.005
C_1               0.003
C_2               0.002
C_10              0.002
C_3               0.000


In [224]:
from sklearn.metrics import f1_score
pred = model.predict(X_test)
print(accuracy_score(y_test,pred))
print(f1_score(y_test, pred, average='binary'))
print(confusion_matrix(y_test, pred))
rf_proba = model.predict_proba(X_test)[:,][:,0]

1.0
1.0
[[48  0]
 [ 0 53]]


In [225]:
gbm_model = GradientBoostingClassifier()
gbm_model.fit(X_train,y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [226]:
pred = gbm_model.predict(X_test)
print(accuracy_score(y_test,pred))
print(f1_score(y_test, pred, average='binary'))
print(confusion_matrix(y_test, pred))
gbm_proba = gbm_model.predict_proba(X_test)[:,][:,0]

1.0
1.0
[[48  0]
 [ 0 53]]


In [227]:
# from sklearn import svm
# param_test1 = {'C':[x for x in range(1,201,50)],'gamma':[0.01,0.005,0.001]}
# # svm_model = GridSearchCV(estimator = svm.SVC(kernel='linear', probability=True),param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
# svm_model = svm.SVC(kernel='linear', C=1, gamma=0.01, probability=True) 
# svm_model.fit(X_train,y_train)
# # print([print(x) for x in svm_model.grid_scores_])
# # print(svm_model.best_params_)
# # print(svm_model.best_score_)

In [228]:
# pred = svm_model.predict(X_test)
# print(accuracy_score(y_test,pred))
# print(f1_score(y_test, pred, average='binary'))
# print(confusion_matrix(y_test, pred))
# svm_proba = svm_model.predict_proba(X_test)[:,][:,0]

In [229]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [230]:
pred = lr_model.predict(X_test)
print(accuracy_score(y_test,pred))
print(f1_score(y_test, pred, average='binary'))
print(confusion_matrix(y_test, pred))
lr_proba = lr_model.predict_proba(X_test)[:,][:,0]

1.0
1.0
[[48  0]
 [ 0 53]]


In [231]:
from sklearn.naive_bayes import GaussianNB
nb_model= GaussianNB()
nb_model.fit(X_train, y_train)

GaussianNB(priors=None)

In [232]:
pred = nb_model.predict(X_test)
print(accuracy_score(y_test,pred))
print(f1_score(y_test, pred, average='binary'))
print(confusion_matrix(y_test, pred))
nb_proba = nb_model.predict_proba(X_test)[:,][:,0]

1.0
1.0
[[48  0]
 [ 0 53]]


In [233]:
test_res_df = pd.DataFrame()
test_res_df["rf"] = rf_proba
test_res_df["gbm"] = gbm_proba
# test_res_df["svm"] = svm_proba
test_res_df["lr"] = lr_proba
test_res_df["nb"] = nb_proba
test_res_df.head()



,rf,gbm,lr,nb
0,1.0,0.999723,0.961100,1.0
1,0.0,0.000312,0.032855,0.0
2,0.0,0.000312,0.048158,0.0
3,0.9,0.999723,0.968317,1.0
4,0.0,0.000312,0.035393,0.0


In [234]:
X_tr, X_te, y_tr, y_te = train_test_split(test_res_df, y_test, train_size=0.5, random_state=0)
param_test1 = {'n_estimators':[x for x in range(20,81,10)],'max_depth':[x for x in range(5,16,2)], 'min_samples_split':[x for x in range(200,1001,200)]}
# test_res_model = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.01, min_samples_split=500,min_samples_leaf=50,max_depth=8,max_features='sqrt',subsample=0.8,random_state=0), 
# param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
test_res_model = GradientBoostingClassifier(learning_rate=0.01, min_samples_split=500,min_samples_leaf=50,max_depth=8,max_features='sqrt',subsample=0.8,random_state=0,n_estimators=20)
test_res_model = GradientBoostingClassifier()
test_res_model.fit(X_tr,y_tr)
# print([print(x) for x in test_res_model.grid_scores_])
# print(test_res_model.best_params_)
# print(test_res_model.best_score_)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [235]:
rf_res = test_res_model.feature_importances_
#show to plot importances
importances = pd.DataFrame({'feature':test_res_df.columns,'importance':np.round(test_res_model.feature_importances_,3)})
importances = importances.sort_values('importance',ascending=False).set_index('feature')
print(importances)

         importance
feature            
rf             0.21
gbm            0.19
lr             0.17
nb             0.17


In [236]:
pred_res = test_res_model.predict(X_te)
print(accuracy_score(y_te,pred_res))
print(f1_score(y_te, pred_res, average='binary'))
print(confusion_matrix(y_te, pred_res))

1.0
1.0
[[19  0]
 [ 0 32]]


In [237]:
n = -1
print(df.iloc[n])
pred_res = gbm_model.predict_proba(X.iloc[n])
print(pred_res)
print(gbm_model.predict(X.iloc[n]))

Date           3-Aug-15
Close              3.18
C_1                1.22
C_2                1.26
C_3                 1.3
C_4                 1.3
C_5                1.33
C_6                1.43
C_7                1.39
C_8                1.48
C_9                1.51
C_10               1.53
Open               3.48
High               3.48
Volume           1.6248
Close_shift        3.23
Shift_ratio     0.98452
Result                0
Name: 504, dtype: object
[[  9.99722606e-01   2.77393759e-04]]
[0]


C:\Program Files\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Program Files\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
